In [10]:
import pandas as pd
import os

# Data processing

In [11]:
Data_Directory = './Data_1'

file = './Data_1/ble_datalog_2024-09-12_16-05-30.csv'

In [12]:
coord_mapping = {
    'ble_datalog_2024-09-12_16-05-30.csv': (1, 1),
    'ble_datalog_2024-09-12_15-57-06.csv': (3, 1),
    'ble_datalog_2024-09-12_15-38-16.csv': (5, 1),
    'ble_datalog_2024-09-12_15-48-01.csv': (7, 1),
    'ble_datalog_2024-09-12_16-15-03.csv': (1, 3),
    'ble_datalog_2024-09-12_16-30-02.csv': (3, 3),
    'ble_datalog_2024-09-12_16-37-48.csv': (5, 3),
    'ble_datalog_2024-09-12_16-47-14.csv': (7, 3),
    'ble_datalog_2024-09-12_17-11-06.csv': (3, 5),
    'ble_datalog_2024-09-12_17-03-48.csv': (5, 5),
    'ble_datalog_2024-09-12_16-54-24.csv': (7, 5)
    # Add more mappings as needed
}


In [13]:
def normalize_time_columns(df, time_columns):
    for col in time_columns:
        df[col] = pd.to_datetime(df[col], format='%H-%M-%S')
        min_time = df[col].min()
        df[col] = (df[col] - min_time).dt.total_seconds()
    return df

# Function to process each CSV file
def process_csv(file_path, x_coord, y_coord):
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file_path)

    # Add X_Coord and Y_Coord from mapping
    data['X_Coord'] = x_coord
    data['Y_Coord'] = y_coord

    # Identify and normalize time columns
    time_columns = [col for col in data.columns if 'Time' in col]
    data = normalize_time_columns(data, time_columns)

    return data

# Main function to process all CSV files in a directory
def combine_all_csvs(directory, coord_mapping):
    combined_data = []  # List to store individual DataFrames
    
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)

            # Get X_Coord and Y_Coord from the mapping
            if filename in coord_mapping:
                x_coord, y_coord = coord_mapping[filename]
            else:
                # Default coordinates if not in the mapping
                x_coord, y_coord = (0, 0)

            # Process CSV and add to the list
            data = process_csv(file_path, x_coord, y_coord)
            combined_data.append(data)

    # Concatenate all DataFrames into one large DataFrame
    combined_df = pd.concat(combined_data, ignore_index=True)
    return combined_df



In [14]:
combined_df = combine_all_csvs(Data_Directory, coord_mapping)


# Now print the DataFrame
combined_df

,Tx_0 RSSI,Tx_0 Batt,Tx_0 Time,Tx_1 RSSI,Tx_1 Batt,Tx_1 Time,Tx_2 RSSI,Tx_2 Batt,Tx_2 Time,Tx_3 RSSI,...,Tx_5 Batt,Tx_5 Time,Tx_6 RSSI,Tx_6 Batt,Tx_6 Time,Tx_7 RSSI,Tx_7 Batt,Tx_7 Time,X_Coord,Y_Coord
0,-63,-2.00,0.0,-66,4.20,0.0,-67,4.17,0.0,-70,...,-2.00,0.0,-74,-2.00,0.0,-76,4.18,0.0,5,1
1,-63,-2.00,0.0,-70,4.19,0.0,-67,-2.00,0.0,-69,...,4.14,0.0,-75,-2.00,0.0,-76,4.18,0.0,5,1
2,-63,4.18,0.0,-67,4.20,0.0,-68,4.16,0.0,-68,...,4.14,0.0,-75,-2.00,0.0,-75,4.19,0.0,5,1
3,-64,4.19,0.0,-69,-2.00,0.0,-70,4.17,0.0,-68,...,-2.00,0.0,-73,4.17,0.0,-75,4.18,0.0,5,1
4,-63,-2.00,0.0,-67,4.19,0.0,-68,4.16,0.0,-69,...,4.16,0.0,-74,-2.00,0.0,-76,4.18,0.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,-73,4.18,0.0,-65,4.19,0.0,-70,4.18,0.0,127,...,4.13,0.0,-65,4.15,0.0,127,-99.00,0.0,3,5
106,-75,4.19,0.0,-64,4.19,0.0,-71,4.16,0.0,-79,...,4.13,0.0,-66,4.16,0.0,127,-99.00,0.0,3,5
107,-73,4.18,0.0,-65,4.18,0.0,-71,4.14,0.0,127,...,4.13,0.0,-66,4.15,0.0,127,-99.00,0.0,3,5
108,-74,-2.00,0.0,-65,4.19,0.0,-71,4.16,0.0,-80,...,4.14,0.0,-67,-2.00,0.0,127,-99.00,0.0,3,5


## Calculating average in RSSI and Batt


In [15]:
def create_avg_df(combined_df):
    # Group by X_Coord and Y_Coord
    combined_df = combined_df.drop([col for col in combined_df.columns if 'Time' in col], axis=1)

    grouped = combined_df.groupby(['X_Coord', 'Y_Coord']).mean().reset_index()

    # Rename columns to reflect that these are averages
    avg_columns = {col: col.replace('RSSI', 'RSSI_Avg').replace('Batt', 'Batt_Avg') for col in grouped.columns if 'RSSI' in col or 'Batt' in col}
    
    # Apply the renaming to the grouped DataFrame
    grouped.rename(columns=avg_columns, inplace=True)
    
    return grouped

In [16]:
avg_df = create_avg_df(combined_df)
avg_df

,X_Coord,Y_Coord,Tx_0 RSSI_Avg,Tx_0 Batt_Avg,Tx_1 RSSI_Avg,Tx_1 Batt_Avg,Tx_2 RSSI_Avg,Tx_2 Batt_Avg,Tx_3 RSSI_Avg,Tx_3 Batt_Avg,Tx_4 RSSI_Avg,Tx_4 Batt_Avg,Tx_5 RSSI_Avg,Tx_5 Batt_Avg,Tx_6 RSSI_Avg,Tx_6 Batt_Avg,Tx_7 RSSI_Avg,Tx_7 Batt_Avg
0,1,1,-61.1,4.184,-78.3,2.949,-74.3,3.547,-78.5,2.318,64.5,-68.672,-75.6,1.683,-71.7,-0.150,127.0,-99.000
1,1,3,-66.8,2.331,-66.7,4.188,-56.5,-6.155,-67.6,2.318,-73.2,1.073,-73.2,2.908,-76.2,2.927,-75.8,2.328
2,3,1,-59.1,2.949,-59.7,2.953,-71.6,1.697,-63.6,2.317,42.7,-58.354,-66.7,1.067,-78.2,0.463,-77.1,4.184
3,3,3,-67.3,2.330,-58.0,2.953,-71.4,2.312,-69.3,1.084,-81.4,2.920,-71.8,3.523,-71.2,0.464,-72.3,2.943
4,3,5,-74.2,2.327,-64.8,2.953,-71.1,4.159,-36.5,-19.552,-71.9,3.528,-63.0,4.132,-65.9,2.927,127.0,-99.000
5,5,1,-63.1,1.711,-68.0,2.953,-68.3,2.312,-68.7,2.316,-18.8,-28.023,-65.3,1.684,-74.2,0.465,-75.8,2.947
6,5,3,-73.9,2.949,-75.0,2.335,-71.1,2.312,-67.4,2.931,-74.8,2.303,-69.6,0.453,-66.6,1.078,105.8,-89.300
7,5,5,-78.2,1.712,-73.2,2.948,-58.8,-8.006,-66.8,2.932,-68.0,1.074,-67.8,2.907,-64.2,3.544,-78.5,2.330
8,7,1,-69.5,2.948,-63.7,3.567,-66.6,4.168,-65.5,2.319,127.0,-99.000,-72.2,2.908,-18.7,-28.636,127.0,-99.000
9,7,3,-70.7,2.944,-69.2,1.710,-72.2,1.696,-61.6,3.548,-80.5,2.916,-68.8,2.910,-68.6,1.695,-77.7,2.952
